In [ ]:
import pandas as pd

df = pd.read_csv('../separate_dfs/premium_card_data.csv')
df['date'] = pd.to_datetime(df['date'])

df

,client_code,name,status,age,city,avg_monthly_balance_KZT,date,data_source,type,category,direction,amount,currency
0,1,Айгерим,Зарплатный клиент,NaN,Алматы,NaN,2025-06-02 12:10:21,transaction,NaN,Продукты питания,NaN,18848.85,KZT
1,1,Айгерим,Зарплатный клиент,NaN,Алматы,NaN,2025-06-04 12:10:35,transaction,NaN,Продукты питания,NaN,12139.34,KZT
2,1,Айгерим,Зарплатный клиент,NaN,Алматы,NaN,2025-06-04 17:00:58,transaction,NaN,Кафе и рестораны,NaN,5635.01,KZT
3,1,Айгерим,Зарплатный клиент,NaN,Алматы,NaN,2025-06-04 17:40:05,transaction,NaN,Кафе и рестораны,NaN,5029.07,KZT
4,1,Айгерим,Зарплатный клиент,NaN,Алматы,NaN,2025-06-04 19:00:08,transaction,NaN,Кафе и рестораны,NaN,15408.79,KZT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,60,Ермек,Зарплатный клиент,NaN,Кызылорда,NaN,2025-08-31 10:00:00,transaction,NaN,Кафе и рестораны,NaN,12000.00,KZT
6376,60,Ермек,Зарплатный клиент,NaN,Кызылорда,NaN,2025-08-31 11:20:39,transaction,NaN,Кафе и рестораны,NaN,3489.42,KZT
6377,60,Ермек,Зарплатный клиент,NaN,Кызылорда,NaN,2025-08-31 12:10:15,transaction,NaN,Продукты питания,NaN,8255.46,KZT
6378,60,Ермек,Зарплатный клиент,NaN,Кызылорда,NaN,2025-08-31 17:00:58,transaction,NaN,Кафе и рестораны,NaN,4951.28,KZT
